In [4]:
!pip install pandas numpy
import pandas as pd
import numpy as np


In [22]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTQEI2bS5E6GBqa9U3cWf9Hddje62AbkGUonNJWLEhKHeBMWJTWZ9u_Mva1dVtPizi6XyKK-h2KGz2P/pub?gid=1691864415&single=true&output=csv"
df = pd.read_csv(
    url,
    delimiter=',',
    engine='python',
    on_bad_lines='skip'
)
df.head()

,order_id,date,customer_name,product,quantity,price,total,city
0,1,2024-01-03,John Doe,Laptop,1,1000,1000,Jakarta
1,2,2024-01-03,Jane Doe,Mouse,2,100,200,Bandung
2,3,2024-01-04,Budi,Keyboard,1,500,500,Jakarta
3,4,2024-01-05,Alice,Monitor,1,200,200,Surabaya
4,5,2024-01-05,John Doe,Laptop,1,1000,1000,Jakarta


In [23]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

df.head()

,order_id,date,customer_name,product,quantity,price,total,city
0,1,2024-01-03,John Doe,Laptop,1,1000,1000,Jakarta
1,2,2024-01-03,Jane Doe,Mouse,2,100,200,Bandung
2,3,2024-01-04,Budi,Keyboard,1,500,500,Jakarta
3,4,2024-01-05,Alice,Monitor,1,200,200,Surabaya
4,5,2024-01-05,John Doe,Laptop,1,1000,1000,Jakarta


In [24]:
def clean_quantity(x):
    if pd.isna(x): return np.nan
    x = str(x).strip()

    # (100) → -100
    if x.startswith("(") and x.endswith(")"):
        return -float(x.replace("(", "").replace(")", "").replace(",", ""))

    # "1,200" → 1200
    x = x.replace(",", "")

    # "-" , "--" → NaN
    if x in ["-", "--", ""]:
        return np.nan

    try:
        return float(x)
    except:
        return np.nan

df["quantity_clean"] = df["quantity"].apply(clean_quantity)
df[["quantity", "quantity_clean"]].head()


,quantity,quantity_clean
0,1,1.0
1,2,2.0
2,1,1.0
3,1,1.0
4,1,1.0


In [25]:
def clean_price(x):
    if pd.isna(x): return np.nan
    x = str(x).replace(",", "").strip()
    try:
        return float(x)
    except:
        return np.nan

df["price_clean"] = df["price"].apply(clean_price)
df[["price", "price_clean"]].head()


,price,price_clean
0,1000,1000.0
1,100,100.0
2,500,500.0
3,200,200.0
4,1000,1000.0


In [26]:
df["date_clean"] = pd.to_datetime(
    df["date"],
    errors="coerce",
    infer_datetime_format=True
)

df[["date", "date_clean"]].head()


/tmp/ipython-input-4247104496.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["date_clean"] = pd.to_datetime(


,date,date_clean
0,2024-01-03,2024-01-03
1,2024-01-03,2024-01-03
2,2024-01-04,2024-01-04
3,2024-01-05,2024-01-05
4,2024-01-05,2024-01-05


In [27]:
df["total"] = df["quantity_clean"] * df["price_clean"]
df[["quantity_clean", "price_clean", "total"]].head()


,quantity_clean,price_clean,total
0,1.0,1000.0,1000.0
1,2.0,100.0,200.0
2,1.0,500.0,500.0
3,1.0,200.0,200.0
4,1.0,1000.0,1000.0


In [28]:
df_clean = df.dropna(subset=["date_clean", "quantity_clean", "price_clean"])
df_clean.shape


(5, 11)

In [29]:
df_clean.head()


,order_id,date,customer_name,product,quantity,price,total,city,quantity_clean,price_clean,date_clean
0,1,2024-01-03,John Doe,Laptop,1,1000,1000.0,Jakarta,1.0,1000.0,2024-01-03
1,2,2024-01-03,Jane Doe,Mouse,2,100,200.0,Bandung,2.0,100.0,2024-01-03
2,3,2024-01-04,Budi,Keyboard,1,500,500.0,Jakarta,1.0,500.0,2024-01-04
3,4,2024-01-05,Alice,Monitor,1,200,200.0,Surabaya,1.0,200.0,2024-01-05
4,5,2024-01-05,John Doe,Laptop,1,1000,1000.0,Jakarta,1.0,1000.0,2024-01-05
